In [1]:
'''
Notebook para asociar una velocidad de viento a una curva de potencia ideal con un rango de error por definir
'''

'\nNotebook para asociar una velocidad de viento a una curva de potencia ideal con un rango de error por definir\n'

# Imports

In [6]:
%matplotlib notebook
import pandas as pd
import numpy as np
#importa las fuciones que cree desde otro notebook (instalar modulo ipynb)
import ipynb.fs.defs.my_funcs_clusters as myfunc
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
from matplotlib import rc, font_manager
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython import display as dpy
import ipynb.fs.defs.my_funcs_clusters as myfunc
import matplotlib

In [3]:
pd.set_option('display.max_rows', 100)
#matplotlib.use('nbAGG')

In [18]:
import importlib
importlib.reload(myfunc)

<module 'ipynb.fs.defs.my_funcs_clusters' (C:\Users\mungu\Documents\GitHub\aero\my_funcs_clusters.ipynb)>

# Procesamiento de datos

In [68]:
data_path ='datos/DP-PB01-2005.xlsx'
mf_pow_curve_path ='datos/power curve GE 2_5 127.csv'
df_wind_dir, df_comp_vel,df_mf_curve = myfunc.proc_dat_bcs\
    (
        data_path,
        mf_pow_curve_path,
        mf_vel_colname='wind_speed(m/s)',
        mf_pw_colname= 'Power(kW)',
        mf_rename_cols=('vel','pw')
    )
# power column comes in kw
df_mf_curve['pw']= df_mf_curve.pw.apply(lambda x: x*1000)


Inciando procesamiento de datos...

Total de registros: 52560
Numero de filas sin datos
day     0.000
hour    0.000
wdir    0.000
vwind   0.000
dtype: float64
Fin del procesamiento de datos


## Interpolación de la curva del fabricante

In [72]:
# puntos de la curva del fabricante
plt.figure(1)
plt.plot(df_mf_curve.index,df_mf_curve.pw)
plt.show()

<IPython.core.display.Javascript object>

In [73]:
v_min =0
#maximo de la curva de potencia del fabricante
v_max = df_mf_curve.index.max()
df_mfcurve_interp = myfunc.interp_pow_curve(df_mf_curve,wind_min_max=(v_min,v_max))
plt.figure(5,figsize=(10,8))
plt.scatter(df_mfcurve_interp.index,df_mfcurve_interp.pw,s=2)
plt.show()

<IPython.core.display.Javascript object>

## Crear un dataframe viento-potencia del fabricante

In [74]:
#voy a usar vwind redondeado a un decimal
#el porcentaje de error lo defino despues
#columna de potencias segun la velocidad del viento
#redondeando los valores de velocidad de viento a un decimal
df_wind_dir['vwind'] = np.round(df_wind_dir.vwind,1)
col_pw_mf = [df_mfcurve_interp[df_mfcurve_interp.index==v].pw.values[0] for
             v in  df_wind_dir.vwind]

#data frame que contiene los datos de viento reales y la potencia
#generada por esos datos de viento si evaluara la curva de
#potencia con ellos
df_wind_pow_ideal = pd.DataFrame(df_wind_dir.vwind)
df_wind_pow_ideal['pw']=col_pw_mf

# Encontrar la relación viento-potencia
Quiero saber por cada x cantidad de viento, cuanta potencia sube

In [75]:
fechaini = '2005-01-01 00:00:00'
fechafin= '2005-01-01 23:59:59'
df_wind_pow_ideal_1Dia = df_wind_pow_ideal.loc[fechaini:fechafin]
df_wind_pow_ideal1Dia_ord=df_wind_pow_ideal_1Dia.sort_values(by=['vwind'])
plt.plot(df_wind_pow_ideal1Dia_ord.vwind,df_wind_pow_ideal1Dia_ord.pw)

In [76]:
#porque son las tres de la mañana, se salta el primer dia
#HORRIBLE CODIGO
rango_fechas = pd.date_range(df_wind_pow_ideal.index[0],df_wind_pow_ideal.index[-1])

###########   widgets #########

sslider =widgets.SelectionSlider(
    options=rango_fechas,
    description='Dia:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

output3= widgets.Output()

display(sslider, output3)

############ funcion y ploteo###############
plt.ioff()
fig = plt.figure(figsize=(10,7))
axes = fig.add_subplot(111)

hfig = display(fig, display_id=True)
def on_value_change3(change,):
    with output3:
        #no sirve si la fecha y hora exacta no existe
        dia=df_wind_pow_ideal.loc[change['new']:change['new']+pd.Timedelta(1,'day')]
        dia_ord = dia.sort_values(by='vwind')
        #ax.set_xdata(dia_ord.vwind)
        #ax.set_ydata(dia_ord.pw)
        axes.cla()
        axes.plot(dia_ord.vwind,dia_ord.pw)
        axes.set_xlim([0,30])
        axes.set_ylim([-40e3,2.2e6])
        fig.canvas.draw()
        hfig.update(fig)

sslider.observe(on_value_change3, names='value')

SelectionSlider(continuous_update=False, description='Dia:', options=(Timestamp('2005-01-01 00:10:00', freq='D…

Output()

<IPython.core.display.Javascript object>

In [77]:
#Recolectando datos
#numero de dias
d1 =df_wind_pow_ideal.index[0]
d2=df_wind_pow_ideal.index[-1]
n =d2-d1
ndias_total=n.days


# Observando la variacion entre velocidades de viento iguales y potencia

In [78]:
#buscar valores unicos de velocidad de viento
vv_unic= np.sort(df_wind_pow_ideal.vwind.unique())
a =vv_unic
vvmin = df_wind_dir.vwind.min()
vvmax = df_wind_dir.vwind.max()
b =np.round(np.arange(vvmin,vvmax,0.1),decimals=1)
c= list(set(a)  ^ set(b))
print('Deberian ser %d velocidades, pero son %d'%(len(b),len(a)))
print('Faltan %d velocidades'%(len(c)))
print('Las velocidades que faltan son:')
print(*c, sep = ", ") 


Deberian ser 180 velocidades, pero son 169
Faltan 13 velocidades
Las velocidades que faltan son:
18.0, 16.4, 16.6, 16.9, 17.1, 17.4, 17.5, 17.9, 16.3, 16.8, 17.3, 17.7, 17.8


In [79]:
#d = df_wind_pow_ideal.loc[df_wind_pow_ideal.vwind==vv_unic[100]]


In [80]:
#porque son las 5:30 de la mañana, se salta el primer dia
#HORRIBLE CODIGO
#no se ordenand las velocidades de viento por potencia,debe ser por hora de lectura

###########   widgets #########

sslidervv =widgets.SelectionSlider(
    options=vv_unic,
    description='Velocidad:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

outputvv= widgets.Output()

display(sslidervv, outputvv)

############ funcion y ploteo###############
plt.ioff()
fig = plt.figure(figsize=(10,7))
axes = fig.add_subplot(111)

hfig = display(fig, display_id=True)
def on_value_changevv(change,):
    with outputvv:
        #no sirve si la fecha y hora exacta no existe
        vv_pw=df_wind_pow_ideal.loc[df_wind_pow_ideal.vwind==change['new']]
        df_describe =vv_pw.describe()
        #convertir el describe a texto y ponerlo en formato cientifico para la tabla
        describe_sci =[[format(element,'.2E') for element in el] for el in df_describe.values.tolist()]

        axes.cla()
        axes.plot(vv_pw.index,vv_pw.pw)
        axes.table(describe_sci, colLabels=df_describe.columns,rowLabels=df_describe.T.columns)
        axes.tick_params(axis='x', pad=-20)
        axes.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        #axes.set_xlim([0,30])
#         axes.set_ylim([-40e3,2.2e6])
        fig.canvas.draw()
        hfig.update(fig)
sslidervv.observe(on_value_changevv, names='value')

SelectionSlider(continuous_update=False, description='Velocidad:', options=(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,…

Output()

<IPython.core.display.Javascript object>

# Ubicar transiciones en curva de potencia

In [81]:
import importlib
importlib.reload(myfunc)

<module 'ipynb.fs.defs.my_funcs_clusters' (C:\Users\mungu\Documents\GitHub\aero\my_funcs_clusters.ipynb)>

## clustering

In [82]:
#####################

n_clusters = 15
n_subclu=0
clusters_wind_pow = myfunc.KMData()
clusters_wind_pow.dataframe_to_cluster(df_comp_vel, n_clusters=n_clusters,n_sub_clusters=n_subclu,
    clusters_data='wind', subclusters_data='pw',dfVP=df_wind_pow_ideal,pw_col_name='pw',
    wind_col_name='vwind')

#def dataframe_to_cluster(dfvxvy, n_clusters,dfVP =None, n_subclu=0, clusters_data=None,subclusters_data=None,
#                         datadir=None):
#crear un dataframe viento, direccion, cluster
#df_wind_pow_cl = df_wind_pow_ideal.copy()
#tomando las labels de kmeans y sumandole uno por que empieza en 0
#ademas agregando la letra C y ordenando de menor v a mayor vel
#col_clnames = ['C' + str(clusters_wind.cl_ord[c]+1) for c in clusters_wind.kmeans_labels]
#df_wind_pow_cl['cluster'] = col_clnames
#del col_clnames



C:\Users\mungu\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Plot

### Plot solo viento

In [83]:
#solo viento
clsclord=('wind','power')
ploti = myfunc.PlotSubClusterInt()
ploti.create_plot(clusters_wind_pow.comp_vel, figsize=(20, 10), idx_centroids=clusters_wind_pow.idx_centroids,fign=208,
                  showlBetz=False, showCent= True, showOpt= 'Numero')
plt.show()

Manufacturer power curve missing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
### Plot viento potencia ideal

In [85]:

#viento potencia
ploti = myfunc.PlotSubClusterInt()
ploti.create_plot(clusters_wind_pow.comp_vel,
                  dfclvp=clusters_wind_pow.dfclvp,
                  datavp =df_wind_pow_ideal,
                  dfMfgCurve = df_mf_curve,
                  cl_scl_order=('wind', 'pow'),
                  figsize=(20, 10),
                  idx_centroids=clusters_wind_pow.idx_centroids,
                  fign=208,
                  showOpt= 'Numero')
plt.show()

AttributeError: 'PlotSubClusterInt' object has no attribute 'dfclvv_in'

# Dispersion de potencia (MAD)

In [86]:
n_clusters = 15
n_subclu=0
clusters_wind_pow = myfunc.KMData()
clusters_wind_pow.dataframe_to_cluster(
    df_comp_vel,
    n_clusters=n_clusters,
    n_sub_clusters=n_subclu,
    clusters_data='wind',
    subclusters_data='pw',
    dfVP=df_wind_pow_ideal,
    pw_col_name='pw',
    wind_col_name='vwind')

C:\Users\mungu\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [87]:
clusters_wind_pow.idx_centroids

,cluster_ID,PCTimeStamp,vx,vy,vwind,pw,mag_windv
cluster_ID_ord,,,,,,,
1,7,2005-01-30 00:40:00,0.047,0.090,0.100,0.000,0.102
2,0,2005-02-27 22:50:00,-0.776,1.805,2.000,0.000,1.965
3,14,2005-05-03 04:20:00,2.244,0.232,2.300,0.000,2.256
4,2,2005-05-15 21:00:00,0.460,-2.750,2.800,0.000,2.788
5,3,2005-11-14 00:30:00,-2.790,3.160,4.200,236000.000,4.215
6,9,2005-09-01 12:30:00,-4.103,-1.084,4.200,236000.000,4.244
7,13,2005-10-03 16:30:00,3.253,5.107,6.100,791000.000,6.055
8,6,2005-12-19 06:00:00,-3.503,5.303,6.400,914000.000,6.356
9,5,2005-11-28 16:40:00,3.283,-5.899,6.800,1097200.000,6.751


In [88]:
#dispersión absoluta media
state_mad = []
for i in range(n_clusters):
    state_mad.append(clusters_wind_pow.dfclvp['C'+str(i+1)].vwind.mad())

C:\Users\mungu\AppData\Local\Temp\ipykernel_26868\1215647689.py:4: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  state_mad.append(clusters_wind_pow.dfclvp['C'+str(i+1)].vwind.mad())


In [89]:
clusters_wind_pow.idx_centroids

,cluster_ID,PCTimeStamp,vx,vy,vwind,pw,mag_windv
cluster_ID_ord,,,,,,,
1,7,2005-01-30 00:40:00,0.047,0.090,0.100,0.000,0.102
2,0,2005-02-27 22:50:00,-0.776,1.805,2.000,0.000,1.965
3,14,2005-05-03 04:20:00,2.244,0.232,2.300,0.000,2.256
4,2,2005-05-15 21:00:00,0.460,-2.750,2.800,0.000,2.788
5,3,2005-11-14 00:30:00,-2.790,3.160,4.200,236000.000,4.215
6,9,2005-09-01 12:30:00,-4.103,-1.084,4.200,236000.000,4.244
7,13,2005-10-03 16:30:00,3.253,5.107,6.100,791000.000,6.055
8,6,2005-12-19 06:00:00,-3.503,5.303,6.400,914000.000,6.356
9,5,2005-11-28 16:40:00,3.283,-5.899,6.800,1097200.000,6.751


In [90]:
df_mf_curve.plot()
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# probabilidad de cada estado

In [91]:
#probabilidades de cada estado
dfclvv = myfunc.create_clustered_data(df_wind_dir,clusters_wind_pow.kmeans_labels)

nregcl=[]
for col in dfclvv.columns:
    nregcl.append(len(dfclvv[dfclvv[col].notna()]))
ntotreg= len(df_wind_dir)
df_prob_x_estado = pd.DataFrame(clusters_wind_pow.idx_centroids['mag_windv'].values,columns=['vel'])
df_prob_x_estado.set_index(dfclvv.columns,inplace=True)
df_prob_x_estado.index.name= 'cluster'
df_prob_x_estado['%prob']=np.array(nregcl)/ntotreg *100
df_prob_x_estado

,vel,%prob
cluster,,
C1,0.102,24.534
C2,1.965,9.572
C3,2.256,6.781
C4,2.788,4.043
C5,4.215,9.178
C6,4.244,2.578
C7,6.055,0.881
C8,6.356,9.083
C9,6.751,3.234


 Además, si agrupamos los clusters conjuntos con velocidades de viento similares y porcentajes de ocurrencia
significativos, podemos agrupar los clusters de la siguiente manera:
<ul>
<li>C1, C2, C3, C4, C5 y C6</li>
<li>C8 y C9 </li>
<li>C10 y C11</li>
<li>C12 y C13</li>
<li>C14 y C15</li>
</ul>

El cluster C7 se elimina debido a la baja probabilidad de ocurrir.



In [92]:
col_to_join = [['C1','C2','C3','C4','C5','C6',],['C8','C9'],['C10','C11'],['C12','C13'],['C14','C15']]
col_to_del = [['C7']]
dfclvv_joined = myfunc.join_clusters(dfclvv.copy(),col_to_join,col_to_del)
df_comp_vel_joined = myfunc.join_clusters(clusters_wind_pow.comp_vel.copy(),col_to_join,col_to_del)
clsclord=('wind',None)
ploti = myfunc.PlotSubClusterInt()
ploti.create_plot(
    df_comp_vel_joined,
    figsize=(20, 10),
    fign='Figure '+ str(plt.gcf().number+1),
    save_folder='figures_tests/',
    filename = 'plot_clusters_joined',
    showlBetz=False,
    showCent= False,
    dfMfgCurve=df_mf_curve
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [93]:
df_comp_vel_joined.columns

MultiIndex([( 'C1', 'vx'),
            ( 'C1', 'vy'),
            ( 'C8', 'vx'),
            ( 'C8', 'vy'),
            ('C10', 'vx'),
            ('C10', 'vy'),
            ('C12', 'vx'),
            ('C12', 'vy'),
            ('C14', 'vx'),
            ('C14', 'vy')],
           )

In [94]:
#get centroids
#select power values with timestamp and compute power centroid (mean)
list_of_clusters_joined = ['C1','C8','C10','C12','C14']
pw_centroids=[]
for c in list_of_clusters_joined:
    joined_clusters_indexes=df_comp_vel_joined[c].dropna().index
    centroid = df_wind_pow_ideal.loc[joined_clusters_indexes].pw.mean()
    pw_centroids.append(centroid)
    print('Power centroid of %s: %0.4f '%(c,centroid))

Power centroid of C1: 77863.7444 
Power centroid of C8: 1015917.7016 
Power centroid of C10: 1513089.5293 
Power centroid of C12: 2180689.1612 
Power centroid of C14: 2477140.2396 


In [95]:
pw_centroids

[77863.74437806269,
 1015917.7015755329,
 1513089.5292987511,
 2180689.161168709,
 2477140.239605356]

In [96]:

nregcl=[]
for col in dfclvv_joined.columns:
    nregcl.append(len(dfclvv_joined[dfclvv_joined[col].notna()]))
ntotreg= len(df_wind_dir)
df_prob_x_estado_pw = pd.DataFrame(pw_centroids,columns=['pw'])
df_prob_x_estado_pw.set_index(dfclvv_joined.columns,inplace=True)
df_prob_x_estado_pw.index.name= 'cluster'
df_prob_x_estado_pw['%prob']=np.array(nregcl)/ntotreg *100
df_prob_x_estado_pw

,pw,%prob
cluster,,
C1,77863.744,56.687
C8,1015917.702,12.318
C10,1513089.529,9.903
C12,2180689.161,12.112
C14,2477140.240,8.099


For two collector circuits and 10 2.5 MW wind turbines for each circuit, we have:

$Base\ state\ total\ power  = 1015917.702W \times 20 \approx 20.31MW   $

Where base state is the state with greater probability (C8).

# Potencias y slider gain version 1 (for grouped states)

In [117]:
df_prob_x_estado_pw.loc['C8'].pw

1015917.7015755329

In [118]:
#Estado  (base) ( no necesariamiente el numero de estado coincide con el de idx_centroids por la agrupacion)
#el estado base es el 10 (fotografía del sistema)
#con slider 0: g1= 0.056, g2=0.061
# power of most probable state, grouped clusters
pw_state_most_prob = df_prob_x_estado_pw.loc['C8'].pw
gWT1_pw = pw_state_most_prob*10 #group of wind turbines 1
gWT2_pw = pw_state_most_prob*10



#base de potencia
potencia_base = gWT1_pw+gWT2_pw #MW
print('potencia g1 %f, potencia g2 %f, potencia g1+g2 (base) %f'%(gWT1_pw,gWT2_pw,potencia_base))

potencia g1 10159177.015755, potencia g2 10159177.015755, potencia g1+g2 (base) 20318354.031511


In [149]:
10159177.015755/1e6

10.159177015754999

In [155]:
#grouped states, with for cycle im changing from grouped state number to a sequential state number
lista_potg1_g2_sg = []#lista (tupla) de (estado,potencia grupo1 WT,potencia grupo2,sg) slider gain
for idx in range(len(df_prob_x_estado_pw)):
    pw  = df_prob_x_estado_pw.iloc[idx].pw
    lista_potg1_g2_sg.append(
        (
            idx+1,
            pw*10,
            pw*10,
            pw*10 + pw*10,
            ((pw*20 - potencia_base) /1e8)/2
        )
    )
    
for i in lista_potg1_g2_sg:
    print('Estado: %i, Potencia g1[Mw]: %f, Potencia g2[Mw]: %f, G1+G2; %f,  Slider Gain: %0.6f'%(i[0],i[1]/1E6,
                                                                                                  i[2]/1E6,
                                                                                                  i[3]/1E6, i[4]))

Estado: 1, Potencia g1[Mw]: 0.778637, Potencia g2[Mw]: 0.778637, G1+G2; 1.557275,  Slider Gain: -0.093805
Estado: 2, Potencia g1[Mw]: 10.159177, Potencia g2[Mw]: 10.159177, G1+G2; 20.318354,  Slider Gain: 0.000000
Estado: 3, Potencia g1[Mw]: 15.130895, Potencia g2[Mw]: 15.130895, G1+G2; 30.261791,  Slider Gain: 0.049717
Estado: 4, Potencia g1[Mw]: 21.806892, Potencia g2[Mw]: 21.806892, G1+G2; 43.613783,  Slider Gain: 0.116477
Estado: 5, Potencia g1[Mw]: 24.771402, Potencia g2[Mw]: 24.771402, G1+G2; 49.542805,  Slider Gain: 0.146122


In [163]:
df_slider_values = pd.DataFrame(lista_potg1_g2_sg,columns= ['Estado','PwG1','PwG2','G1+G2','Slider'])
df_slider_values.set_index('Estado',inplace=True)
df_slider_values.to_csv('states_power_output.csv')
df_slider_values

,PwG1,PwG2,G1+G2,Slider
Estado,,,,
1,778637.444,778637.444,1557274.888,-0.094
2,10159177.016,10159177.016,20318354.032,0.000
3,15130895.293,15130895.293,30261790.586,0.050
4,21806891.612,21806891.612,43613783.223,0.116
5,24771402.396,24771402.396,49542804.792,0.146


# Potencias y slider gain version 2
Necesario ejecutar version 1. resumen de version 1.


In [ ]:
'''
Datos de respaldo

Estado: 1 velocidad 0.10 m/s = 0.000000E+00 w
Estado: 2 velocidad 2.00 m/s = 0.000000E+00 w
Estado: 3 velocidad 2.30 m/s = 0.000000E+00 w
Estado: 4 velocidad 2.80 m/s = 0.000000E+00 w
Estado: 5 velocidad 4.20 m/s = 9.920000E+04 w
Estado: 6 velocidad 4.20 m/s = 9.920000E+04 w
Estado: 7 velocidad 6.10 m/s = 4.696000E+05 w
Estado: 8 velocidad 6.40 m/s = 5.674000E+05 w
Estado: 9 velocidad 6.80 m/s = 7.086000E+05 w
Estado: 10 velocidad 7.50 m/s = 9.600000E+05 w
Estado: 11 velocidad 7.60 m/s = 9.974000E+05 w
Estado: 12 velocidad 8.90 m/s = 1.497200E+06 w
Estado: 13 velocidad 10.30 m/s = 1.888800E+06 w
Estado: 14 velocidad 11.10 m/s = 2.000000E+06 w
Estado: 15 velocidad 11.20 m/s = 2.000000E+06 w

'''

In [55]:
#calcular las potencias base (slider = 0)
#esta potencia es de las granjas, tomada directamente del scope de simulink
pot_base_g1 = 6.1048e-2   #tomado del raw de psse
pot_base_g2 = 5.6352e-2

#base de potencia
potencia_base = pot_base_g1+pot_base_g2 #MW
print('potencia g1 %f, potencia g2 %f, potencia g1+g2 (base) %f'%(pot_base_g1,pot_base_g2,potencia_base))


potencia g1 0.061048, potencia g2 0.056352, potencia g1+g2 (base) 0.117400


In [56]:
#potencia por cada aerogenerador
g1_gen_ind = pot_base_g1/13
g2_gen_ind = pot_base_g2/12
print(g1_gen_ind,g2_gen_ind)

0.004696 0.004696


In [74]:
# en watts (MW)
potencia_base*1e8


11740000.0

In [86]:
potencia_base

0.1174

In [85]:
idx=0
-(potencia_base-((lista_pw[idx]*13)+(lista_pw[idx]*12))/1e8)

-0.1174

In [93]:
#para calcular el valor del slider solo tomare en cuenta la potencia más alta
# a partir de ahí la otra granja se va a ajustar con el poweroffset que es 24/26
#-pot_base_g1 +(lista_pw[idx]*13)/1e8))
# esto no lo uso, funcionaria para potencias cero pero no para las demas
# porque una granja tiene un generador mas que la otra asi que la desviacion
# debe de afectar diferente a cada granja, con el offset, la afecta igual
#y se supone que el viento es el que hace que la potencia de salida cambie...
# no hay diferencia de viento en las dos granjas

lista_potg1_g2_sg = []#lista (tupla) de (estado,potencia granja1,potencia granja2,sg) slider gain
for idx,ne in enumerate(range(len(lista_estados))):
    lista_potg1_g2_sg.append((lista_estados[idx],lista_pw[idx]*13,lista_pw[idx]*12,lista_pw[idx]*13+lista_pw[idx]*12,
                              -(potencia_base-((lista_pw[idx]*13)+(lista_pw[idx]*12))/1e8)/2))
for i in lista_potg1_g2_sg:
    print('Estado: %i, Potencia g1[Mw]: %f, Potencia g2[Mw]: %f, G1+G2; %f,  Slider Gain: %0.6f'%(i[0],i[1]/1E6,
                                                                                                  i[2]/1E6,
                                                                                                  i[3]/1E6, i[4]))

Estado: 1, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: -0.058700
Estado: 2, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: -0.058700
Estado: 3, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: -0.058700
Estado: 4, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: -0.058700
Estado: 5, Potencia g1[Mw]: 1.289600, Potencia g2[Mw]: 1.190400, G1+G2; 2.480000,  Slider Gain: -0.046300
Estado: 6, Potencia g1[Mw]: 1.289600, Potencia g2[Mw]: 1.190400, G1+G2; 2.480000,  Slider Gain: -0.046300
Estado: 7, Potencia g1[Mw]: 6.104800, Potencia g2[Mw]: 5.635200, G1+G2; 11.740000,  Slider Gain: -0.000000
Estado: 8, Potencia g1[Mw]: 7.376200, Potencia g2[Mw]: 6.808800, G1+G2; 14.185000,  Slider Gain: 0.012225
Estado: 9, Potencia g1[Mw]: 9.211800, Potencia g2[Mw]: 8.503200, G1+G2; 17.715000,  Slider Gain: 0.029875
Estado: 10, Potencia g1[Mw]: 12.480000, Poten

In [44]:
df_mfcurve_interp

,pw
0.000,0.000
0.100,0.000
0.200,0.000
0.300,0.000
0.400,0.000
...,...
24.500,2000000.000
24.600,2000000.000
24.700,2000000.000
24.800,2000000.000


# Guardar en csv las potencias y los valores del slider

In [94]:
df_pot_slider= pd.DataFrame(lista_potg1_g2_sg,columns=['Estado','PwG1','PwG2','G1+G2','Slider'])
df_pot_slider.set_index('Estado',inplace=True)
df_pot_slider

,PwG1,PwG2,G1+G2,Slider
Estado,,,,
1,0.000,0.000,0.000,-0.059
2,0.000,0.000,0.000,-0.059
3,0.000,0.000,0.000,-0.059
4,0.000,0.000,0.000,-0.059
5,1289600.000,1190400.000,2480000.000,-0.046
6,1289600.000,1190400.000,2480000.000,-0.046
7,6104800.000,5635200.000,11740000.000,-0.000
8,7376200.000,6808800.000,14185000.000,0.012
9,9211800.000,8503200.000,17715000.000,0.030


In [95]:
df_pot_slider.to_csv('states_power_output.csv')

In [ ]:

#estado 14, slider gain
#potencia en estado 14,8.2ms 9.446000E+05 w * n aerogeneradores

pwE14g1=26E+06
pwE14g2=24E+06

#potencia base-potencia de la granja1 en el estado 14, 8.2
SGE14=(potencia_base*1E6-(pwE14g1+pwE14g2))/200E6 #SG control slider gain de simulink (desviación de potencia)
print('Potencia E14 granja 1: %f\nPotencia E14 granja 2: %f\nDesviacion de potencia SG: %0.6f'%(pwE14g1,pwE14g2,SGE14))

In [ ]:
#estado 14, slider gain
#potencia en estado 14,8.2ms 9.446000E+05 w * n aerogeneradores
pwE14g1=9.446000E+05*25
pwE14g2=9.446000E+05*23

#potencia base-potencia de la granja1 en el estado 14, 8.2
SGE14=(potencia_base*1E6-pwE14g1)/200E6 #SG control slider gain de simulink (desviación de potencia)
print('Potencia E14 granja 1: %f\nPotencia E14 granja 2: %f\nDesviacion de potencia SG: %f'%(pwE14g1,pwE14g2,SGE14))

In [ ]:
#estado 9, slider gain
#potencia en estado 9,5.3m/s 279915.905 * n aerogeneradores
pwE9g1=2.432000E+05*13
pwE9g2=2.432000E+05*12
SGE9=(potencia_base*1E6-(pwE9g1+pwE9g2))/200E6 #SG control slider gain de simulink (desviación de potencia)

#potencia base-potencia de la granja1 en el estado 14, 8.2
print('Potencia E9 granja 1: %f\nPotencia E9 granja 2: %f\nDesviacion de potencia SG: %f'%(pwE9g1,pwE9g2,SGE9))